In [ ]:
import zipfile
import os

# Ruta del archivo ZIP
zip_path = "/content/chincheData_FinalA.zip"  # Ruta al archivo ZIP (ajústalo si está en otra ubicación)
extract_path = "/content/extracted_files"  # Carpeta donde se extraerán los archivos

# Extraer el archivo ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Archivo extraído correctamente en: {extract_path}")

# Mostrar la estructura de las carpetas extraídas
for root, dirs, files in os.walk(extract_path):
    level = root.replace(extract_path, "").count(os.sep)
    indent = " " * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = " " * 4 * (level + 1)
    for file in files:
        print(f"{sub_indent}{file}")


Archivo extraído correctamente en: /content/extracted_files
extracted_files/
    chincheData_Final/
        chincheData/
            bestAumentado.pt
            modeloCelu.py
            best.pt
            modelo.py
            best_Exp.pt
            dataset.yaml
            train/
                images/
                    imagen_135.jpg
                    imagen_179.jpg
                    imagen_22.jpg
                    imagen_143.jpg
                    imagen_131.jpg
                    imagen_221.jpg
                    imagen_62.jpg
                    imagen_105.jpg
                    imagen_218.jpg
                    imagen_64.jpg
                    imagen_89.jpg
                    imagen_26.jpg
                    imagen_161.jpg
                    imagen_219.jpg
                    imagen_24.jpg
                    imagen_147.jpg
                    imagen_217.jpg
                    imagen_202.jpg
                    imagen_18.jpg
                    imagen_207.j

# Sección nueva

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import os
from tqdm import tqdm
import shutil

# Directorios
input_images_dir = "./extracted_files/chincheData/train/images"  # Carpeta de imágenes originales
input_labels_dir = "./extracted_files/chincheData/train/labels"  # Carpeta de etiquetas originales (.txt)
output_images_dir = "./extracted_files/chincheData/augmented_train/images"
output_labels_dir = "./extracted_files/chincheData/augmented_train/labels"

# Crear carpetas destino
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)

# Transformaciones de augmentación
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.Blur(blur_limit=3, p=0.2),
    A.RandomResizedCrop(640, 640, scale=(0.8, 1.0), ratio=(0.75, 1.33), p=0.5)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# Procesar imágenes y etiquetas
for img_name in tqdm(os.listdir(input_images_dir)):
    if not img_name.endswith(('.jpg', '.png', '.jpeg')):
        continue

    # Cargar imagen y su archivo de etiquetas
    img_path = os.path.join(input_images_dir, img_name)
    label_path = os.path.join(input_labels_dir, img_name.replace('.jpg', '.txt'))

    if not os.path.exists(label_path):
        continue

    image = cv2.imread(img_path)
    h, w = image.shape[:2]

    # Leer bounding boxes del archivo .txt
    with open(label_path, "r") as f:
        bboxes = []
        class_labels = []
        for line in f.readlines():
            cls, x_center, y_center, bbox_w, bbox_h = map(float, line.strip().split())
            bboxes.append([x_center, y_center, bbox_w, bbox_h])
            class_labels.append(cls)

    # Aplicar transformaciones
    for i in range(5):  # Generar 5 variaciones por imagen
        augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        aug_image = augmented['image']
        aug_bboxes = augmented['bboxes']

        # Guardar imagen aumentada
        aug_img_name = f"{img_name.split('.')[0]}_aug{i}.jpg"
        aug_img_path = os.path.join(output_images_dir, aug_img_name)
        cv2.imwrite(aug_img_path, aug_image)

        # Guardar etiquetas aumentadas
        aug_label_path = os.path.join(output_labels_dir, aug_img_name.replace('.jpg', '.txt'))
        with open(aug_label_path, "w") as f:
            for bbox, cls in zip(aug_bboxes, class_labels):
                f.write(f"{int(cls)} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")


  4%|▍         | 4/91 [00:00<00:10,  8.37it/s]/usr/local/lib/python3.10/dist-packages/albumentations/core/bbox_utils.py:478: RuntimeWarning: invalid value encountered in divide
  & (clipped_box_areas / denormalized_box_areas >= min_visibility - epsilon)
100%|██████████| 91/91 [00:13<00:00,  6.86it/s]


In [ ]:
import albumentations as A
import cv2
import os
from tqdm import tqdm

# Directorios
input_images_dir = "./extracted_files/chincheData/val/images"   # Carpeta de imágenes originales para test
input_labels_dir = "./extracted_files/chincheData/val/labels"   # Carpeta de etiquetas originales (.txt)
output_images_dir = "./extracted_files/chincheData/augmented_/images"  # Carpeta destino para imágenes aumentadas
output_labels_dir = "./extracted_files/chincheData/augmented_test/labels"  # Carpeta destino para etiquetas aumentadas

# Crear carpetas destino
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)

# Transformaciones de augmentación
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.Blur(blur_limit=3, p=0.2),
    A.RandomResizedCrop(640, 640, scale=(0.8, 1.0), ratio=(0.75, 1.33), p=0.5)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'], min_visibility=0.3))

# Procesar imágenes y etiquetas
for img_name in tqdm(os.listdir(input_images_dir)):
    if not img_name.endswith(('.jpg', '.png', '.jpeg')):
        continue

    # Cargar imagen y su archivo de etiquetas
    img_path = os.path.join(input_images_dir, img_name)
    label_path = os.path.join(input_labels_dir, img_name.replace('.jpg', '.txt'))

    if not os.path.exists(label_path):
        print(f"Etiqueta no encontrada para {img_name}, saltando...")
        continue

    image = cv2.imread(img_path)

    # Leer bounding boxes del archivo .txt
    with open(label_path, "r") as f:
        bboxes = []
        class_labels = []
        for line in f.readlines():
            cls, x_center, y_center, bbox_w, bbox_h = map(float, line.strip().split())
            bboxes.append([x_center, y_center, bbox_w, bbox_h])
            class_labels.append(cls)

    # Aplicar transformaciones
    for i in range(5):  # Generar 5 variaciones por imagen
        augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)

        # Filtrar bounding boxes válidos
        if len(augmented['bboxes']) == 0:
            continue  # Saltar si no hay cajas válidas después de la transformación

        aug_image = augmented['image']
        aug_bboxes = augmented['bboxes']

        # Guardar imagen aumentada
        aug_img_name = f"{img_name.split('.')[0]}_aug{i}.jpg"
        aug_img_path = os.path.join(output_images_dir, aug_img_name)
        cv2.imwrite(aug_img_path, aug_image)

        # Guardar etiquetas aumentadas
        aug_label_path = os.path.join(output_labels_dir, aug_img_name.replace('.jpg', '.txt'))
        with open(aug_label_path, "w") as f:
            for bbox, cls in zip(aug_bboxes, class_labels):
                f.write(f"{int(cls)} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")

print("Imágenes y etiquetas aumentadas para el conjunto de test se han generado correctamente.")


 93%|█████████▎| 25/27 [00:02<00:00, 10.14it/s]


ValueError: Expected y_min for bbox [   0.068481 -5.0664e-07     0.94833     0.99812           0] to be in the range [0.0, 1.0], got -5.066394805908203e-07.

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 48.1 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

# Download the pre-trained model if it doesn't exist
from pathlib import Path
if not Path('YOLOv8l.pt').is_file():
    !yolo export model=yolov8l.pt format=pt  # Download YOLOv8l.pt

# Load pre-trained model
model = YOLO('yolov8m.pt')  # Load the downloaded model

# Train the model
model.train(data="./extracted_files/chincheData/dataset.yaml", epochs=100, imgsz=640, batch=10)

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 972, in entrypoint
    getattr(model, mode)(**overrides)  # default args from model
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/model.py", line 738, in export
    return Exporter(overrides=args, _callbacks=self.callbacks)(model=self.model)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/exporter.py", line 192, in __call__
    raise ValueError(f"Invalid export format='{fmt}'. Valid formats are {fmts}")
ValueError: Invalid export format='pt'. Valid formats are ('torchscript', 'onnx', 'openvino', 'engine', 'coreml', 'saved_model', 'pb', 'tflite', 'edgetpu', 'tfjs', 'paddle', 'mnn', 'ncnn', 'imx')
Sentry is attempting to send 2 pending events
Waiting up to 2 seconds
Press Ctrl-C to quit
Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4,

train: Scanning /content/extracted_files/chincheData/augmented_train/labels... 455 images, 0 backgrounds, 0 corrupt: 100%|██████████| 455/455 [00:00<00:00, 1351.35it/s]

train: New cache created: /content/extracted_files/chincheData/augmented_train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/extracted_files/chincheData/augmented_val/labels... 125 images, 0 backgrounds, 0 corrupt: 100%|██████████| 125/125 [00:00<00:00, 411.93it/s]

val: New cache created: /content/extracted_files/chincheData/augmented_val/labels.cache


...

                   all        125        125      0.917      0.904      0.905      0.609



100 epochs completed in 0.588 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 52.0MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:03<00:00,  2.12it/s]


                   all        125        125      0.873      0.934       0.94      0.628
Speed: 0.3ms preprocess, 10.2ms inference, 0.0ms loss, 4.2ms postprocess per image
Results saved to runs/detect/train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78a284757f10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
from ultralytics import YOLO

# Download the pre-trained model if it doesn't exist
from pathlib import Path
if not Path('YOLOv8l.pt').is_file():
    !yolo export model=yolov8l.pt format=pt  # Download YOLOv8l.pt

# Load pre-trained model
model = YOLO('yolov8m.pt')  # Load the downloaded model

# Train the model
# Entrenamiento con aumentación de datos integrada
model.train(
    data="./extracted_files/chincheData/dataset.yaml",
    epochs=100,
    imgsz=640,
    batch=10,
    augment=True,  # Activa augmentación de YOLOv8
    degrees=10,    # Rotación (grados)
    translate=0.1, # Traslación
    scale=0.5,     # Zoom
    shear=10,      # Deformación (shear)
    flipud=0.5,    # Volteo vertical
    fliplr=0.5     # Volteo horizontal
)


Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 972, in entrypoint
    getattr(model, mode)(**overrides)  # default args from model
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/model.py", line 738, in export
    return Exporter(overrides=args, _callbacks=self.callbacks)(model=self.model)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/exporter.py", line 192, in __call__
    raise ValueError(f"Invalid export format='{fmt}'. Valid formats are {fmts}")
ValueError: Invalid export format='pt'. Valid formats are ('torchscript', 'onnx', 'openvino', 'engine', 'coreml', 'saved_model', 'pb', 'tflite', 'edgetpu', 'tfjs', 'paddle', 'mnn', 'ncnn', 'imx')
Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=./extracted_file

train: Scanning /content/extracted_files/chincheData/train/labels.cache... 91 images, 0 backgrounds, 0 corrupt: 100%|██████████| 91/91 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/extracted_files/chincheData/val/labels.cache... 27 images, 0 backgrounds, 0 corrupt: 100%|██████████| 27/27 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.00046875), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         27         27      0.928       0.96      0.944       0.55



100 epochs completed in 0.175 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train/weights/best.pt, 52.0MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.91it/s]


                   all         27         27      0.961      0.904      0.959      0.654
Speed: 0.2ms preprocess, 33.9ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78a1a4565060>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 